In [1]:
import os
from pathlib import Path

In [2]:
notebooks_path = Path.cwd()
repo_path = notebooks_path.parent
os.chdir(str(notebooks_path))
#print current working directory
print(notebooks_path)

/home/ricardino/Documents/MAIA/tercer_semestre/Extra_activities/INCan/Added_value_feature_uncertainty/notebooks
